In [1]:
import pandas as pd
from rsome import ro
from rsome import grb_solver as grb

## Data Ingestion

In [2]:
separation_time = pd.read_csv("data - Separation Time.csv",header=None).values
separation_time

array([[99999,     3,    15,    15,    15,    15,    15,    15,    15,
           15,     3,     3,    15,    15,     3],
       [    3, 99999,    15,    15,    15,    15,    15,    15,    15,
           15,     3,     3,    15,    15,     3],
       [   15,    15, 99999,     8,     8,     8,     8,     8,     8,
            8,    15,    15,     8,     8,    15],
       [   15,    15,     8, 99999,     8,     8,     8,     8,     8,
            8,    15,    15,     8,     8,    15],
       [   15,    15,     8,     8, 99999,     8,     8,     8,     8,
            8,    15,    15,     8,     8,    15],
       [   15,    15,     8,     8,     8, 99999,     8,     8,     8,
            8,    15,    15,     8,     8,    15],
       [   15,    15,     8,     8,     8,     8, 99999,     8,     8,
            8,    15,    15,     8,     8,    15],
       [   15,    15,     8,     8,     8,     8,     8, 99999,     8,
            8,    15,    15,     8,     8,    15],
       [   15,    15,   

In [3]:
landing_time_cost_df = pd.read_csv("data - Landing Time.csv")
landing_time_cost_df

,Earliest Landing Time,Target Landing Time,Latest Landing Time,penalty cost per unit of time for landing before target,penalty cost per unit of time for landing after target,Size
0,129,155,559,10,10,3
1,190,250,732,10,10,3
2,84,93,501,30,30,2
3,89,98,509,30,30,2
4,100,111,536,30,30,2
5,107,120,552,30,30,2
6,109,121,550,30,30,1
7,109,120,544,30,30,1
8,115,128,557,30,30,1
9,134,151,610,30,30,1


In [5]:
earliest_landing = landing_time_cost_df["Earliest Landing Time"].values
target_landing = landing_time_cost_df["Target Landing Time"].values
latest_landing = landing_time_cost_df["Latest Landing Time"].values

early_landing_penalty = landing_time_cost_df["penalty cost per unit of time for landing before target"].values
late_landing_penalty = landing_time_cost_df["penalty cost per unit of time for landing after target"].values
aircraft_size = landing_time_cost_df["Size"].values

In [7]:
N = len(earliest_landing)
N

15

In [8]:
runway_max_size = pd.read_csv("data - Runway.csv")['Max Size'].values
runway_max_size

array([2, 1, 1, 1, 3, 3, 1, 2, 1, 3, 2, 1, 1, 3, 3], dtype=int64)

In [9]:
num_runways = runway_max_size.shape[0]
num_runways

15

## Model

In [10]:
model = ro.Model("Aircaft Landing")

### Decision Variables

In [11]:
scheduled_landing = model.dvar(N)
landing_earliness = model.dvar(N)
landing_lateness = model.dvar(N)

#NxN Matrix to determine whether plane i lands before plane j, i != j
i_lands_before_j = model.dvar((N,N), "B")
# for i in range(N):
#     model.st(i_lands_before_j[i:i] == 0)

# NxN Matrix to determine whether plane i lands before plane j, i != j
lands_on_same_runway = model.dvar((N,N), "B")
# for i in range(N):
#     model.st(lands_on_same_runway[i:i] == 0)

# Nxnum_runways Matrix to determine whether plane i lands on runway r
runway_allocation = model.dvar((N,num_runways), "B")

### Objective Function

In [12]:
model.min(sum(early_landing_penalty*landing_earliness) + sum(late_landing_penalty*landing_lateness))

### Constraints

In [13]:
# Aircraft has to land between the latest and earliest possible time
model.st(earliest_landing <= scheduled_landing <= latest_landing)

# Calculating the earliness & lateness
# Since landing_earliness, landing_lateness > 0
# If scheduled_landing - target_landing >= 0, landing_earliness >=0 & landing_lateness ==0
# If scheduled_landing - target_landing <= 0, landing_earliness ==0 & landing_lateness >=0
model.st(scheduled_landing - target_landing == landing_earliness - landing_lateness)

# If 2 planes are scheduled for the same runway, then at least S amt of time should have elapsed
# between plane i and plane j
M = 99999
for j in range(N):
    for i in range(N):
        if i != j:
            model.st(scheduled_landing[j] - scheduled_landing[i] >= 
                     separation_time[i,j]*lands_on_same_runway[i,j] - M*i_lands_before_j[j,i])
            
# Every 2 plane must land either before or after and not the same time
for j in range(N):
    for i in range(N):
        if i != j:
            model.st(i_lands_before_j[i,j] + i_lands_before_j[j,i] == 1)

# Defining the link between lands_on_same_runway & runway_allocation
for j in range(N):
    for i in range(N):
        if i != j:
            for r in range(num_runways):
                model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)

# # Aircraft size should not be larger than max size of runway
# for aircraft_index in range(N):
#     for runway_index in range(num_runways):
#         if aircraft_size[aircraft_index] > runway_max_size[runway_index]:
#             model.st(runway_allocation[aircraft_index,runway_index] == 0)

# Each aircraft can only land on 1 runway
for i in range(N):
    model.st(sum(runway_allocation[i,r] for r in range(num_runways)) == 1)

# Non negative Constraints
model.st(scheduled_landing >= 0,
        landing_earliness >= 0,
        landing_lateness >= 0
        )

### Solving

In [14]:
model.solve(grb)

Restricted license - for non-production use only - expires 2024-10-28
Being solved by Gurobi...


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [13]:
print("Scheduled landing times:", scheduled_landing.get())
print("Earliness:", landing_earliness.get())
print("Lateness:", landing_lateness.get())
print("Whether i lands before j:", i_lands_before_j.get())
print("Whether i lands on the same runway as j:", lands_on_same_runway.get())
print("Runway allocation:", runway_allocation.get())
print("Total cost:", model.get())

Scheduled landing times: [155. 258.  98. 106. 123. 132. 138. 140. 150. 180.]
Earliness: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lateness: [0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]
Whether i lands before j: [[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Whether i lands on the same runway as j: [[0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]
Runway allocation: [[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]

# Wrapping the Model as a Function

In [14]:
def get_optimal_landing(no_of_runways, separation_time, earliest_landing, target_landing, latest_landing, 
                        early_penalty, late_penalty):
    '''
    Prints the optimal landing time and runway allocation for each plane, among other variables.
    
    Parameters:
        no_of_runways (int): the number of available runways
        separation_time (numpy.ndarray): a matrix of separation times when planes land on the same runway
        earliest_landing (numpy.ndarray): an array of the earliest time each plane can land
        target_landing (numpy.ndarray): an array of the target landing time for each plane
        latest_landing (numpy.ndarray): an array of the latest time each plane can land
        early_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane early
        late_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane late
    '''
    
    check_input_shapes(separation_time, len(earliest_landing), len(target_landing), len(latest_landing), 
                       len(early_penalty), len(late_penalty))
    
    no_of_planes = len(earliest_landing)
    
    model = ro.Model('Airplane Landing')
    
    # Decision variables
    scheduled_landing_time = model.dvar(no_of_planes)
    earliness = model.dvar(no_of_planes)
    lateness = model.dvar(no_of_planes)
    lands_before = model.dvar((no_of_planes, no_of_planes), "B")
    lands_on_same_runway = model.dvar((no_of_planes, no_of_planes), "B")
    runway_allocation = model.dvar((no_of_planes, no_of_runways), "B")
    
    # Objective
    model.min(sum(early_penalty*earliness) + sum(late_penalty*lateness))
    
    # Constraints
    model.st(earliest_landing <= scheduled_landing_time <= latest_landing)
    model.st(scheduled_landing_time - target_landing == earliness - lateness)
    M = 99999
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                model.st(scheduled_landing_time[j] - scheduled_landing_time[i] >= 
                         separation_time[i,j]*lands_on_same_runway[i,j] - M*lands_before[j,i])
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                model.st(lands_before[i,j] + lands_before[j,i] == 1)
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                for r in range(no_of_runways):
                    model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)
    for i in range(no_of_planes):
        model.st(sum(runway_allocation[i,r] for r in range(no_of_runways)) == 1)
    
    model.st(scheduled_landing_time >= 0,
             earliness >= 0,
             lateness >= 0)
    
    # Get solution
    model.solve(grb)
    
    # Print results
    print("-------------------------")
    print("Scheduled landing times:", scheduled_landing_time.get())
    print("Earliness:", earliness.get())
    print("Lateness:", lateness.get())
    print("Whether i lands before j:\n", lands_before.get())
    print("Whether i lands on the same runway as j:\n", lands_on_same_runway.get())
    print("Runway allocation:\n", runway_allocation.get())
    
    landing_time_to_plane = {v:k for k, v in enumerate(scheduled_landing_time.get())}
    print("Sequence of landing:", [landing_time_to_plane[time] for time in sorted(scheduled_landing_time.get())])
    print("Sequence of landing times:", sorted(scheduled_landing_time.get()))
    print("Total cost:", model.get())
    
    
def check_input_shapes(separation, *args):
    assert len(set(args)) == 1, "Length of inputs must be equal"
    assert separation.shape == (args[0], args[0]), "Shape of separation must be NxN, where N is the length of each of the other inputs"

In [15]:
get_optimal_landing(num_runways, separation_time, earliest_landing, target_landing, latest_landing, early_landing_penalty, late_landing_penalty)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0380s
-------------------------
Scheduled landing times: [155. 258.  98. 106. 123. 132. 138. 140. 150. 180.]
Earliness: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lateness: [0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]
Whether i lands before j:
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Whether i lands on the same runway as j:
 [[0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]
Runway allocat